This is a port of https://github.com/wimvanderbauwhede/HaskellMOOC/tree/master/ParsecTutorial discussed in section 4.6 of FL. Add a note here

This is another cell

In [1]:
import Text.ParserCombinators.Parsec
import qualified Text.ParserCombinators.Parsec.Token as P
import Text.ParserCombinators.Parsec.Language
import Data.List ( intercalate )
-- import ShowParser ( parseShow )

In [2]:
parseTest (char 'b') "cons"

parse error at (line 1, column 1):
unexpected "c"
expecting "b"

In [3]:
 parseTest (char 'c') "cons"

'c'

In [6]:
xml_header =  "<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n"

parseShow :: String -> String
parseShow = \str -> xml_header++(run_parser showParser str)

run_parser :: Parser a -> String -> a
run_parser p str =  case parse p "" str of
    Left err -> error $ "parse error at " ++ (show err)
    Right val  -> val  

otag t = "<"++t++">"
ctag t = "</"++t++">"
tag t v = concat [otag t,v,ctag t]

tagAttrs t attrs v = concat [otag (unwords $ [t]++(map (\(k,v) -> concat [k,"=\"",v,"\""]) attrs)),v,ctag t]

joinNL ls = intercalate "\n" ls


showParser :: Parser String 
showParser =
    list_parser <|> -- [ ... ]
    tuple_parser <|> -- ( ... )
    try record_parser <|> -- MkRec { ... }
    adt_parser <|> -- MkADT ...
    number <|>    -- signed integer
    quoted_string <?> "Parse error"

quoted_string = do
    s <- stringLiteral
    return $ "\""++s++"\""

number = do
    n <- integer
    return $ show n

list_parser = do
    ls <- brackets $ commaSep showParser 
    return $ tag "list" $ joinNL $ map (tag "list-elt") ls
    
tuple_parser = do
    ls <- parens $ commaSep showParser 
    return $ tag "tuple" $ unwords $ map (tag "tuple-elt") ls

record_parser = do
    ti <- type_identifier
    ls <- braces $ commaSep kvparser
    return $ tagAttrs "record" [("name",ti)] (joinNL ls)

adt_parser = do
    ti <- type_identifier 
    return $ tag "adt" ti

kvparser = do
    k <- identifier
    symbol "="
    t <- showParser
    return $ tagAttrs "elt" [("key",k)] t

type_identifier = do
    fst <- oneOf ['A' .. 'Z']
    rest <- many alphaNum
    whiteSpace
    return $ fst:rest



lexer       = P.makeTokenParser emptyDef

parens          = P.parens lexer    
brackets        = P.brackets lexer    
braces          = P.braces lexer    
commaSep        = P.commaSep lexer
whiteSpace      = P.whiteSpace lexer    
symbol          = P.symbol lexer    
identifier      = P.identifier lexer
integer         = P.integer lexer    
stringLiteral   = P.stringLiteral lexer 

Line 1: Use camelCase
Found:
xml_header = ...
Why not:
xmlHeader = ...Line 4: Redundant lambda
Found:
parseShow = \ str -> xml_header ++ (run_parser showParser str)
Why not:
parseShow str = xml_header ++ (run_parser showParser str)Line 4: Redundant bracket
Found:
xml_header ++ (run_parser showParser str)
Why not:
xml_header ++ run_parser showParser strLine 6: Use camelCase
Found:
run_parser :: Parser a -> String -> a
Why not:
runParser :: Parser a -> String -> aLine 7: Use camelCase
Found:
run_parser p str = ...
Why not:
runParser p str = ...Line 8: Redundant bracket
Found:
"parse error at " ++ (show err)
Why not:
"parse error at " ++ show errLine 15: Use :
Found:
[t] ++ (map (\ (k, v) -> concat [k, "=\"", v, "\""]) attrs)
Why not:
t : (map (\ (k, v) -> concat [k, "=\"", v, "\""]) attrs)Line 15: Redundant bracket
Found:
[t] ++ (map (\ (k, v) -> concat [k, "=\"", v, "\""]) attrs)
Why not:
[t] ++ map (\ (k, v) -> concat [k, "=\"", v, "\""]) attrsLine 17: Eta reduce
Found:
joinNL ls = intercalate "\n" ls
Why not:
joinNL = intercalate "\n"Line 29: Use camelCase
Found:
quoted_string = ...
Why not:
quotedString = ...Line 33: Use <$>
Found:
do n <- integer
   return $ show n
Why not:
do show <$> integerLine 37: Use camelCase
Found:
list_parser = ...
Why not:
listParser = ...Line 41: Use camelCase
Found:
tuple_parser = ...
Why not:
tupleParser = ...Line 45: Use camelCase
Found:
record_parser = ...
Why not:
recordParser = ...Line 50: Use camelCase
Found:
adt_parser = ...
Why not:
adtParser = ...Line 50: Use <$>
Found:
do ti <- type_identifier
   return $ tag "adt" ti
Why not:
do tag "adt" <$> type_identifierLine 54: Use <$>
Found:
do k <- identifier
   symbol "="
   t <- showParser
   return $ tagAttrs "elt" [("key", k)] t
Why not:
do k <- identifier
   symbol "="
   tagAttrs "elt" [("key", k)] <$> showParserLine 60: Use camelCase
Found:
type_identifier = ...
Why not:
typeIdentifier = ...

In [7]:
data PersonRecord  = MkPersonRecord {
    name :: String,
    address :: Address,
    id :: Integer,
    labels :: [Label]    
     } deriving (Show)

data Address = MkAddress {
    line1 :: String,
    number :: Integer,
    street :: String,
    town :: String,
    postcode :: String
     } deriving (Show)

data Label = Green | Red | Blue | Yellow deriving (Show)

rec1 = MkPersonRecord 
    "Wim Vanderbauwhede" 
    (MkAddress "School of Computing Science" 17 "Lilybank Gdns" "Glasgow" "G12 8QQ")
    557188
    [Green, Red]

rec2 = MkPersonRecord 
    "Jeremy Singer" 
    (MkAddress "School of Computing Science" 17 "Lilybank Gdns" "Glasgow" "G12 8QQ")
    42
    [Blue, Yellow]

-- main = putStrLn $ show [rec1,rec2]    

rec_str =  show [rec1,rec2]    
main = putStrLn $ parseShow rec_str

Line 32: Use camelCase
Found:
rec_str = ...
Why not:
recStr = ...

In [8]:
main

<?xml version="1.0" encoding="UTF-8"?>
<list><list-elt><record name="MkPersonRecord"><elt key="name">"Wim Vanderbauwhede"</elt>
<elt key="address"><record name="MkAddress"><elt key="line1">"School of Computing Science"</elt>
<elt key="number">17</elt>
<elt key="street">"Lilybank Gdns"</elt>
<elt key="town">"Glasgow"</elt>
<elt key="postcode">"G12 8QQ"</elt></record></elt>
<elt key="id">557188</elt>
<elt key="labels"><list><list-elt><adt>Green</adt></list-elt>
<list-elt><adt>Red</adt></list-elt></list></elt></record></list-elt>
<list-elt><record name="MkPersonRecord"><elt key="name">"Jeremy Singer"</elt>
<elt key="address"><record name="MkAddress"><elt key="line1">"School of Computing Science"</elt>
<elt key="number">17</elt>
<elt key="street">"Lilybank Gdns"</elt>
<elt key="town">"Glasgow"</elt>
<elt key="postcode">"G12 8QQ"</elt></record></elt>
<elt key="id">42</elt>
<elt key="labels"><list><list-elt><adt>Blue</adt></list-elt>
<list-elt><adt>Yellow</adt></list-elt></list></elt></recor